In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import tree
from mlxtend.classifier import StackingClassifier
from collections import defaultdict
import math
import re

# Load data

In [3]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

# Data description

In [4]:
train_df.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [5]:
train_df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test_df.describe()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,...,28000.000000,28000.000000,28000.000000,28000.000000,28000.000000,28000.0,28000.0,28000.0,28000.0,28000.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.164607,0.073214,0.028036,0.011250,0.006536,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.473293,3.616811,1.813602,1.205211,0.807475,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,253.000000,254.000000,193.000000,187.000000,119.000000,0.0,0.0,0.0,0.0,0.0


In [7]:
test_df

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Prepare data for algorithms

In [8]:

# X and Y are the input and output of the classifier algorithm
train_y = train_df.label.astype(int).values

# test_x and train_x must have the same number of columns and
# test_x has no "Survived" column so we must drop it from train_x
train_x = train_df.drop("label", axis=1).values

test_x = test_df.values

# Base classification functions

In [9]:
class ClassifierRunner(object):
    
    def __init__(self, pipeline, parameters, debug=False):
        self.pipeline = pipeline
        self.parameters = parameters
        self.grid_search = GridSearchCV(self.pipeline, self.parameters, cv=5)
        self.debug = debug
        
    def fit(self, X, y):
        self.grid_search.fit(X, y)

    @property
    def best_params(self):
        return self.grid_search.best_params_
        
    def get_scores(self, X, y, num_folds=5):
        scores = cross_val_score(self.grid_search.best_estimator_, X, y, cv=num_folds)
        return scores
    
    @property
    def feature_importances(self):
        classifier_step_index = 0
        for step_name, step_process in self.grid_search.best_estimator_.steps:
            if step_name == "classifier":
                break
            classifier_step_index += 1
        feature_importances = self.grid_search.best_estimator_.steps[classifier_step_index][1].feature_importances_
        return sorted(zip(feature_importances, selected_features), reverse=True)
    
    def predict(self, X_test):
        prediction = self.grid_search.predict(X_test)
        return prediction

    def apply_predicition_to_df(self, X_test, output_filename):
        prediction = self.predict(X_test)
        ids = np.arange(1,28001)
        submission = pd.DataFrame( {'ImageId': ids, 'Label': prediction})
        submission.to_csv(output_filename, index=False)
        

# Centroid

In [ ]:
# Basic decision tree Pipeline
pipeline = Pipeline([
        ('minmaxscaler', StandardScaler()),
        ('classifier', NearestCentroid())
    ])

parameters = {
    'classifier__metric': ["manhattan", "euclidean"], 
    'classifier__shrink_threshold': [None, .05, .1, .2, .5, .55, .6, .7, .74, .75, .77, .8, .85, .9, 1],
}

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, output_filename="results/test_classified_with_nearest_centroid.csv")


# Basic decision tree

In [ ]:
# Basic decision tree Pipeline
pipeline = Pipeline([
        ("pca", PCA()),
        ('classifier', tree.DecisionTreeClassifier())
    ])

max_depths = [None]
parameters = {
    'pca__n_components': [20],
    'classifier__max_depth': max_depths, 
    'classifier__criterion': ["gini"],
    'classifier__splitter': ["random"],
    'classifier__min_samples_split':[2, 3, 4],
    'classifier__random_state': [1],
    'classifier__max_features': [None]
}

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, output_filename="results/test_classified_with_dt.csv")

# MLPClassifier

In [ ]:
# Basic NN classifier Pipeline
pipeline = Pipeline([
        ('minmaxscaler', scaler()),
        ('classifier', MLPClassifier())
    ])

hidden_layer_sizes = [100, 100, 100]
parameters = {
    'classifier__hidden_layer_sizes': hidden_layer_sizes, 
    'classifier__activation': ["identity", "logistic", "tanh", "relu"],
    'classifier__solver': ["lbfgs", "sgd", "adam"],
    'classifier__learning_rate' : ["constant", "invscaling", "adaptive"]
}

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, output_filename="results/test_classified_with_nn.csv")

# Predict test values with KNN

In [ ]:
# KNN Pipeline
pipeline = Pipeline([
        #('minmaxscaler', scaler()),
        ('classifier', KNeighborsClassifier())
    ])

k_values = [1, 2, 3, 4, 5, 7, 10, 15, 20]
parameters = {
    'classifier__n_neighbors': k_values, 
    'classifier__weights': ["uniform", "distance"],
    'classifier__algorithm': ['kd_tree']
}

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, output_filename="results/test_classified_with_knn.csv")

# Predict test values with RandomForest

In [ ]:
# Random forest Pipeline

pipeline = Pipeline([
        ('pca', PCA()),
        ('classifier', RandomForestClassifier())
    ])


parameters = {
                 'pca__n_components': [10, 100],
                 'classifier__max_depth' : [None],
                 'classifier__criterion': ['gini'],
                 'classifier__n_estimators': [100, 300],
                 'classifier__max_features': [None],
                 'classifier__min_samples_split': [2, 3, 10],
                 'classifier__min_samples_leaf': [1, 3, 10],
                 'classifier__bootstrap': [True],
                 'classifier__n_jobs': [-1]
                 }

classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, output_filename="results/test_classified_with_rf.csv")


# Stacking

In [14]:

random_seed = 1

clf1 = KNeighborsClassifier()
clf2 = RandomForestClassifier()
clf3 = GradientBoostingClassifier()
clf4 = GaussianNB()

clf5 = MLPClassifier(
    hidden_layer_sizes = (300, 300),
    alpha = 0.5,
    activation="relu",
    solver="lbfgs",
    random_state=random_seed
)

lr = LogisticRegression()

sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4, clf5], 
                          meta_classifier=lr, use_probas=True)

pipeline = Pipeline([
    ('pca', PCA()),
    ("classifier", sclf)  
])

pca_n_components = 75

parameters = {
    'pca__n_components': [pca_n_components]
}


classifier = ClassifierRunner(pipeline=pipeline, parameters=parameters)

classifier.fit(train_x, train_y)

print ("Best parameters found: ")
print (classifier.best_params)

scores = classifier.get_scores(train_x, train_y)
print ("Expected performance: {:.2f}% (+/-{:.2f}).".format(np.mean(scores)*100., np.std(scores)*100.))

classifier.apply_predicition_to_df(test_x, output_filename="results/test_classified_with_stacking_pca_{}.csv".format(pca_n_components))


Best parameters found: 
{'pca__n_components': 75}
Expected performance: 96.37% (+/-0.12).
